### DS 3002: Final Lab

##### Section 1: Installing and Importing
In this section, we will install and import all necessary functions for the rest of the code.

In [0]:
%pip install pyspark
%pip install findspark
%pip install pymongo[srv]
%pip install sqlalchemy

Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import os
import json
import csv
import pymongo
import pyspark.pandas as pd
import pandas as pd
import requests
import requests.exceptions
import findspark
findspark.add_packages('mysql:mysql-connector-java:8.0.11')
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext.getOrCreate();
spark = SparkSession(sc)
from sqlalchemy import create_engine
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, BinaryType
from pyspark.sql.types import ByteType, ShortType, IntegerType, LongType, FloatType, DecimalType

##### Section 2: Connecting to Azure Database
In this section of the code, we will establish a connection to the database wide-world-importers which is hosted on a server I created in Azure. Afterwards, we will run a few queries and create one of the required dimension tables to properly view and analyze the data.

In [0]:
%scala
Class.forName("com.microsoft.sqlserver.jdbc.SQLServerDriver")

res2: Class[_] = class com.microsoft.sqlserver.jdbc.SQLServerDriver

In [0]:
jdbcHostname = "ds-final.database.windows.net"
jdbcDatabase = "wide-world-importers"
jdbcPort = 1433
jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2}".format(jdbcHostname, jdbcPort, jdbcDatabase) 

connectionProperties = {
  "user" : "haleyeah",
  "password" : "Lh09308!",
  "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

In [0]:
sql_query = """
(SELECT c.CustomerID
    , c.NameStyle
    , c.Title
    , c.FirstName
    , c.MiddleName
    , c.LastName
    , c.Suffix
    , c.CompanyName
    , c.SalesPerson
    , c.EmailAddress
    , c.Phone
    , ca.AddressType
    , a.AddressLine1
    , a.AddressLine2
    , a.City
    , a.StateProvince
    , a.CountryRegion
    , a.PostalCode
    , c.rowguid
    , c.ModifiedDate
FROM [SalesLT].[Customer] AS c
INNER JOIN [SalesLT].[CustomerAddress] AS ca
ON ca.CustomerID = c.CustomerID
INNER JOIN [SalesLT].[Address] AS a
ON a.AddressID = ca.AddressID) customer_dim
"""

df = spark.read.jdbc(url=jdbcUrl, table=sql_query, properties=connectionProperties)
display(df)

CustomerID,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,CompanyName,SalesPerson,EmailAddress,Phone,AddressType,AddressLine1,AddressLine2,City,StateProvince,CountryRegion,PostalCode,rowguid,ModifiedDate
29485,false,Ms.,Catherine,R.,Abel,null,Professional Sales and Service,adventure-works\linda3,catherine0@adventure-works.com,747-555-0171,Main Office,57251 Serene Blvd,null,Van Nuys,California,United States,91411,392AE773-D7EC-48AC-B8D0-6E65B770285C,2009-05-16T16:33:33.077+0000
29486,false,Ms.,Kim,null,Abercrombie,null,Riders Company,adventure-works\jillian0,kim2@adventure-works.com,334-555-0137,Main Office,Tanger Factory,null,Branch,Minnesota,United States,55056,2A5ABA2E-8DB0-4856-A773-21D185F1679E,2009-05-16T16:33:33.077+0000
29489,false,Ms.,Frances,B.,Adams,null,Area Bike Accessories,adventure-works\shu0,frances0@adventure-works.com,991-555-0183,Main Office,6900 Sisk Road,null,Modesto,California,United States,95354,C353FE38-6147-40A3-944D-3736F6297B8C,2009-05-16T16:33:33.090+0000
29490,false,Ms.,Margaret,J.,Smith,null,Bicycle Accessories and Kits,adventure-works\david8,margaret0@adventure-works.com,959-555-0151,Main Office,Lewiston Mall,null,Lewiston,Idaho,United States,83501,7F2D6183-6AEE-4AD1-973D-F45A19B70BF7,2009-05-16T16:33:33.107+0000
29492,false,Mr.,Jay,null,Adams,null,Valley Bicycle Specialists,adventure-works\jillian0,jay1@adventure-works.com,158-555-0142,Main Office,Blue Ridge Mall,null,Kansas City,Missouri,United States,64106,7A2EB695-B6B9-48D4-BAD2-BD33E8CE8CA1,2009-05-16T16:33:33.123+0000
29494,false,Mr.,Samuel,N.,Agcaoili,null,Vinyl and Plastic Goods Corporation,adventure-works\josé1,samuel0@adventure-works.com,554-555-0110,Main Office,No. 25800-130 King Street West,null,Toronto,Ontario,Canada,M4B 1V5,66D2DC13-4220-43B8-B2C7-131183CAEC34,2005-09-01T00:00:00.000+0000
29496,false,Mr.,Robert,E.,Ahlering,null,Fun Toys and Bikes,adventure-works\shu0,robert1@adventure-works.com,678-555-0175,Main Office,6500 East Grant Road,null,Tucson,Arizona,United States,85701,44E51EC2-9D0B-40CC-B725-1A3E81DF9519,2007-09-01T00:00:00.000+0000
29497,false,Mr.,François,null,Ferrier,null,Great Bikes,adventure-works\david8,françois1@adventure-works.com,571-555-0128,Main Office,Eastridge Mall,null,Casper,Wyoming,United States,82601,EEEF3C65-08F9-49C6-AE12-A2FCDEDACAF5,2005-07-01T00:00:00.000+0000
29499,false,Ms.,Amy,E.,Alberts,null,Valley Toy Store,adventure-works\josé1,amy1@adventure-works.com,727-555-0115,Main Office,252851 Rowan Place,null,Richmond,British Columbia,Canada,V6B 3P7,29CD70BE-D5E5-4A7A-A1D5-521BCCE2193D,2006-09-01T00:00:00.000+0000
29502,false,Mr.,Paul,L.,Alcorn,null,Major Sport Suppliers,adventure-works\david8,paul2@adventure-works.com,331-555-0162,Main Office,White Mountain Mall,null,Rock Springs,Wyoming,United States,82901,F193D485-3A5A-415A-9D3E-4A5DB8B4A2A0,2007-07-01T00:00:00.000+0000


In [0]:
%sql
CREATE TABLE IF NOT EXISTS Product_Category
USING org.apache.spark.sql.jdbc
OPTIONS (
  url "jdbc:sqlserver://ds-final.database.windows.net:1433;database=wide-world-importers",
  dbtable "SalesLT.ProductCategory",
  user "haleyeah",
  password "Lh09308!"
)

In [0]:
%sql
SELECT * FROM Product_Category

ProductCategoryID,ParentProductCategoryID,Name,rowguid,ModifiedDate
1,null,Bikes,CFBDA25C-DF71-47A7-B81B-64EE161AA37C,2002-06-01T00:00:00.000+0000
2,null,Components,C657828D-D808-4ABA-91A3-AF2CE02300E9,2002-06-01T00:00:00.000+0000
3,null,Clothing,10A7C342-CA82-48D4-8A38-46A2EB089B74,2002-06-01T00:00:00.000+0000
4,null,Accessories,2BE3BE36-D9A2-4EEE-B593-ED895D97C2A6,2002-06-01T00:00:00.000+0000
5,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2002-06-01T00:00:00.000+0000
6,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2002-06-01T00:00:00.000+0000
7,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2002-06-01T00:00:00.000+0000
8,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2002-06-01T00:00:00.000+0000
9,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2002-06-01T00:00:00.000+0000
10,2,Brakes,D43BA4A3-EF0D-426B-90EB-4BE4547DD30C,2002-06-01T00:00:00.000+0000


##### Section 3: Connecting to MongoDB
For this section of coding, we will establish a connection to MongoDB and access a csv file which was originally obtained from Kaggle (https://www.kaggle.com/datasets/gregorut/videogamesales?select=vgsales.csv).

In [0]:
host_name = "localhost"
ports = {"mongo" : 27017, "mysql" : 3306}

user_id = "root"
password = "Lh09308!"

scr_dbname = "vgsales"
dst_dbname = "vgsales"

In [0]:
def get_sql_dataframe(user_id, pwd, host_name, db_name, sql_query):
    
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    
    conn = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, conn);
    conn.close()
    
    return dframe

def get_mongo_dataframe(user_id, pwd, host_name, port, db_name, collection, query):
    
    if user_id and pwd:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db_name)
        client = pymongo.MongoClient(mongo_uri)
    else:
        conn_str = f"mongodb://{host_name}:{port}/"
        client = pymongo.MongoClient(conn_str)
    
    db = client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query))) #passing in query
    dframe.drop(['_id'], axis=1, inplace=True)
    client.close()
    
    return dframe

def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()
    
def conJSON(csvFile, jsonFile):
    info = {}
    with open(csvFile, encoding='windows-1252') as csvf:
        csvReader = csv.DictReader(csvf)
    for row in csvReader:
        key = row['Row ID']
        info[key] = row
    with open(jsonFile, 'w', encoding='windows-1252') as jsonf:
        jsonf.write(json.dumps(info, indent=4))

In [0]:
port = ports["mongo"]
conn_str = f"mongodb://{host_name}:{port}/"
client = pymongo.MongoClient(conn_str)
db = client["vgsales"]

In [0]:
dbutils.fs.ls("FileStore/shared_uploads/lsh5gny@virginia.edu/vgsales.csv")
df1 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/lsh5gny@virginia.edu/vgsales-1.csv")

In [0]:
df1.head(1)

Out[74]: [Row(Rank='1', Name='Wii Sports', Platform='Wii', Year='2006', Genre='Sports', Publisher='Nintendo', NA_Sales='41.49', EU_Sales='29.02', JP_Sales='3.77', Other_Sales='8.46', Global_Sales='82.74')]

In [0]:
query = {}
port = ports["mongo"]
vgsales = "vgsales"
src_dbname = "vgsales"

df_vgsales = get_mongo_dataframe(None, None, host_name, port, src_dbname, vgsales, query)
df_vgsales.head(2)

---------------------------------------------------------------------------
ServerSelectionTimeoutError               Traceback (most recent call last)
<command-4027744843105956> in <module>
      4 src_dbname = "vgsales"
      5 
----> 6 df_vgsales = get_mongo_dataframe(None, None, host_name, port, src_dbname, vgsales, query)
      7 df_vgsales.head(2)

<command-4027744843105952> in get_mongo_dataframe(user_id, pwd, host_name, port, db_name, collection, query)
     20 
     21     db = client[db_name]
---> 22     dframe = pd.DataFrame(list(db[collection].find(query))) #passing in query
     23     dframe.drop(['_id'], axis=1, inplace=True)
     24     client.close()

/local_disk0/.ephemeral_nfs/envs/pythonEnv-d6111678-d9bf-4a28-83c5-f08e5f0c49cb/lib/python3.8/site-packages/pymongo/cursor.py in next(self)
   1246         if self.__empty:
   1247             raise StopIteration
-> 1248         if len(self.__data) or self._refresh():
   1249             return self.__data.popleft()
   12

##### Section 4: Connecting to an API
In this section, a currency API will be established and updated daily in order to illustrate the currency rates between several different countries and the United States. This API type was necessary as the data used in this project deals with sales in a variety of other countries as well.

In [0]:
def get_api_response(url, response_type):
        response = requests.get(url)
        response.raise_for_status()

In [0]:
url = "https://www.freeforexapi.com/api/live"
{
    "supportedPairs":[
        "EURUSD",
        "EURGBP",
        "GBPUSD",
        "USDJPY",
        "AUDUSD",
        "USDCHF",
        "NZDUSD",
        "USDCAD",
        "USDZAR"
    ],
    "message":"'pairs' parameter is required",
    "code":1001
}
response_type = ['json', 'dataframe']
apicurrency = get_api_response(url, response_type[0])

##### Section 5: Creating Two More Dimesion Tables
Although one additional dimension table was created above, in this section, we will create two additional dimesion tables in Azure and the SQL server which was previously created in order to illustrate the implimentation of the new data.

In [0]:
%sql
CREATE TABLE IF NOT EXISTS AddressDim
USING org.apache.spark.sql.jdbc
OPTIONS (
  url "jdbc:sqlserver://ds-final.database.windows.net:1433;database=wide-world-importers",
  dbtable "SalesLT.Address",
  user "haleyeah",
  password "Lh09308!"
)

In [0]:
%sql
SELECT * FROM AddressDim

AddressID,AddressLine1,AddressLine2,City,StateProvince,CountryRegion,PostalCode,rowguid,ModifiedDate
9,8713 Yosemite Ct.,null,Bothell,Washington,United States,98011,268AF621-76D7-4C78-9441-144FD139821A,2006-07-01T00:00:00.000+0000
11,1318 Lasalle Street,null,Bothell,Washington,United States,98011,981B3303-ACA2-49C7-9A96-FB670785B269,2007-04-01T00:00:00.000+0000
25,9178 Jumping St.,null,Dallas,Texas,United States,75201,C8DF3BD9-48F0-4654-A8DD-14A67A84D3C6,2006-09-01T00:00:00.000+0000
28,9228 Via Del Sol,null,Phoenix,Arizona,United States,85004,12AE5EE1-FC3E-468B-9B92-3B970B169774,2005-09-01T00:00:00.000+0000
32,26910 Indela Road,null,Montreal,Quebec,Canada,H1Y 2H5,84A95F62-3AE8-4E7E-BBD5-5A6F00CD982D,2006-08-01T00:00:00.000+0000
185,2681 Eagle Peak,null,Bellevue,Washington,United States,98004,7BCCF442-2268-46CC-8472-14C44C14E98C,2006-09-01T00:00:00.000+0000
297,7943 Walnut Ave,null,Renton,Washington,United States,98055,52410DA4-2778-4B1D-A599-95746625CE6D,2006-08-01T00:00:00.000+0000
445,6388 Lake City Way,null,Burnaby,British Columbia,Canada,V5A 3A6,53572F25-9133-4A8B-A065-102FF35416EE,2006-09-01T00:00:00.000+0000
446,52560 Free Street,null,Toronto,Ontario,Canada,M4B 1V7,801A1DFC-5125-486B-AA84-CCBD2EC57CA4,2005-08-01T00:00:00.000+0000
447,22580 Free Street,null,Toronto,Ontario,Canada,M4B 1V7,88CEE379-DBB8-433B-B84E-A35E09435500,2006-08-01T00:00:00.000+0000


In [0]:
%sql
CREATE TABLE IF NOT EXISTS OrderDetailDim
USING org.apache.spark.sql.jdbc
OPTIONS (
  url "jdbc:sqlserver://ds-final.database.windows.net:1433;database=wide-world-importers",
  dbtable "SalesLT.SalesOrderDetail",
  user "haleyeah",
  password "Lh09308!"
)

In [0]:
%sql
SELECT * FROM OrderDetailDim

SalesOrderID,SalesOrderDetailID,OrderQty,ProductID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
71774,110562,1,836,356.8980,0.0000,356.898000,E3A1994C-7A68-4CE8-96A3-77FDD3BBD730,2008-06-01T00:00:00.000+0000
71774,110563,1,822,356.8980,0.0000,356.898000,5C77F557-FDB6-43BA-90B9-9A7AEC55CA32,2008-06-01T00:00:00.000+0000
71776,110567,1,907,63.9000,0.0000,63.900000,6DBFE398-D15D-425E-AA58-88178FE360E5,2008-06-01T00:00:00.000+0000
71780,110616,4,905,218.4540,0.0000,873.816000,377246C9-4483-48ED-A5B9-E56F005364E0,2008-06-01T00:00:00.000+0000
71780,110617,2,983,461.6940,0.0000,923.388000,43A54BCD-536D-4A1B-8E69-24D083507A14,2008-06-01T00:00:00.000+0000
71780,110618,6,988,112.9980,0.4000,406.792800,12706FAB-F3A2-48C6-B7C7-1CCDE4081F18,2008-06-01T00:00:00.000+0000
71780,110619,2,748,818.7000,0.0000,1637.400000,B12F0D3B-5B4E-4F1F-B2F0-F7CDE99DD826,2008-06-01T00:00:00.000+0000
71780,110620,1,990,323.9940,0.0000,323.994000,F117A449-039D-44B8-A4B2-B12001DACC01,2008-06-01T00:00:00.000+0000
71780,110621,1,926,149.8740,0.0000,149.874000,92E5052B-72D0-4C91-9A8C-42591803667E,2008-06-01T00:00:00.000+0000
71780,110622,1,743,809.7600,0.0000,809.760000,8BD33BED-C4F6-4D44-84FB-A7D04AFCD794,2008-06-01T00:00:00.000+0000


##### Section 6: Creating a Fact Table
In this section, we will create a fact table.

In [0]:
%sql
CREATE TABLE fact_table
(
SalesOrderID INT,
CountryRegion STRING,
Name STRING
)

##### Section 7: Data Exportation and Visualizations
In this section, we will export the data to Excel and create visualizations in order to properly analyze the data. These are added in the github link.